In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.nn.functional as F
import shutil
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt
import cv2
import gradio as gr
import torch
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from scipy.ndimage import distance_transform_edt
import cv2


Home Page

In [71]:
import gradio as gr

def home_page():
    return gr.Markdown(
        """
        # Cell Segmentor Dashboard
        Welcome to the **Cell Segmentor Dashboard**. This dashboard leverages the power of a U-Net++ model for cell nuclei segmentation, refined through a custom-designed pipeline for image analysis. Built specifically for microscopy imaging in cell research, this tool aids in precise identification and separation of individual nuclei, even in densely packed cellular structures.

        ### Key Features
        - **Automated Nuclei Segmentation**: Upload a microscopy image, and the dashboard will automatically apply the U-Net++ model to detect and segment cell nuclei with high precision.
        - **Refined Separation of Overlapping Nuclei**: Using watershed and distance transform methods, overlapping nuclei are carefully separated for individual analysis.
        - **Interactive Nuclei Properties**: After segmentation, each nucleus is assigned a unique label, and users can explore detailed properties, including area, perimeter, compactness, eccentricity, solidity, and orientation.
        - **Manual Drawing and Property Calculation**: For customized analysis, use the sketchpad tool to draw shapes on a transparent layer over an uploaded image and calculate shape properties.

        ### Data and Training
        - The segmentation model was trained on preprocessed microscopy images and labels from the **Data Science Bowl 2018** dataset, which provides robust, realistic examples of cellular structures.
        - The dataset was split into training, validation, and test sets to ensure model generalization and accuracy across unseen data.

        ### How to Use the Dashboard
        - **Application**: Navigate to the **Segment Cells** tab to start with automated segmentation or to manually draw and analyze shapes.
        - **About**: Visit this section to learn more about the underlying techniques and intended use cases.

        ### Navigation
        - **Segment Cells**: Begin your segmentation task by uploading an image for automated analysis.
        - **Draw and Calculate Shape Properties**: Customize the analysis by drawing regions on an image for specific property calculations.

        ### Acknowledgments
        This tool was developed using U-Net++ architecture and leverages advanced image processing techniques to support scientific research in cellular biology. It is especially suited for applications in cell structure analysis, allowing for accurate morphological insights.

        """
    )

# Create the home page layout
home_interface = gr.Interface(
    fn=home_page,
    inputs=[],
    outputs="markdown",
    title="Cell Segmentor Dashboard - Home"
)

home_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91c9c7e1ba897fbaa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Application Page

In [83]:
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from skimage.morphology import h_maxima
from scipy.ndimage import distance_transform_edt, binary_fill_holes
import cv2
import io
from PIL import Image

# Define U-Net++ model architecture
class UNetPlusPlus(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetPlusPlus, self).__init__()

        # Encoder path
        self.encoder1 = self.double_conv(in_channels, 64)
        self.encoder2 = self.double_conv(64, 128)
        self.encoder3 = self.double_conv(128, 256)
        self.encoder4 = self.double_conv(256, 512)

        # Bottleneck
        self.bottleneck = self.double_conv(512, 1024)

        # Decoder path with nested convolutions (UNet++)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4_1 = self.double_conv(1024, 512)
        self.decoder4_2 = self.double_conv(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3_1 = self.double_conv(512, 256)
        self.decoder3_2 = self.double_conv(512, 256)
        self.decoder3_3 = self.double_conv(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2_1 = self.double_conv(256, 128)
        self.decoder2_2 = self.double_conv(256, 128)
        self.decoder2_3 = self.double_conv(256, 128)
        self.decoder2_4 = self.double_conv(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1_1 = self.double_conv(128, 64)
        self.decoder1_2 = self.double_conv(128, 64)
        self.decoder1_3 = self.double_conv(128, 64)
        self.decoder1_4 = self.double_conv(128, 64)
        self.decoder1_5 = self.double_conv(128, 64)

        # Output layer
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def double_conv(self, in_channels, out_channels, dropout_prob=0.1):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
            nn.Dropout(dropout_prob),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ELU(inplace=True),
        )

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        b = self.bottleneck(F.max_pool2d(e4, 2))

        d4_1 = self.upconv4(b)
        d4_1 = torch.cat([d4_1, e4], dim=1)
        d4_1 = self.decoder4_1(d4_1)

        d3_1 = self.upconv3(d4_1)
        d3_1 = torch.cat([d3_1, e3], dim=1)
        d3_1 = self.decoder3_1(d3_1)

        d2_1 = self.upconv2(d3_1)
        d2_1 = torch.cat([d2_1, e2], dim=1)
        d2_1 = self.decoder2_1(d2_1)

        d1_1 = self.upconv1(d2_1)
        d1_1 = torch.cat([d1_1, e1], dim=1)
        d1_1 = self.decoder1_1(d1_1)

        out = torch.sigmoid(self.out_conv(d1_1))
        out = F.interpolate(out, size=(x.size(2), x.size(3)), mode='bilinear', align_corners=False)
        return out

# Load the pre-trained model weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNetPlusPlus(in_channels=3, out_channels=1).to(device)
model_path = '/content/drive/MyDrive/CellSeg/unet++_224_model_500epochs.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Function for refined cell segmentation
def segment_cells(image):
    image_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float().to(device)
    with torch.no_grad():
        predicted_mask = torch.sigmoid(model(image_tensor)).squeeze().cpu().numpy() > 0.5

    # Fill holes in the binary mask
    filled_mask = binary_fill_holes(predicted_mask).astype(np.uint8)

    # Compute the distance transform
    distance_map = distance_transform_edt(filled_mask)

    # Detect peaks using H-maxima transform
    h_maxima_markers = h_maxima(distance_map, h=0.1 * distance_map.max())
    markers = label(h_maxima_markers)

    # Apply watershed segmentation
    initial_labels = watershed(-distance_map, markers, mask=filled_mask)

    # Refine segmentation for overlapping nuclei
    final_labels = np.zeros_like(initial_labels)
    current_label = 1

    for region in regionprops(initial_labels):
        nucleus_mask = (initial_labels == region.label)
        distance_local = distance_transform_edt(nucleus_mask)
        local_h_maxima = h_maxima(distance_local, h=0.3 * distance_local.max())
        local_markers = label(local_h_maxima)
        refined_labels = watershed(-distance_local, local_markers, mask=nucleus_mask)
        refined_labels[refined_labels > 0] += current_label - 1
        final_labels += refined_labels
        current_label = final_labels.max() + 1

    # Visualization of refined segmentation (color mask only)
    fig_colored, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(final_labels, cmap='nipy_spectral')
    ax.axis('off')
    plt.close(fig_colored)

    # Visualization of numbered binary mask
    fig_numbered, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(filled_mask, cmap='gray')
    for region in regionprops(final_labels):
        y, x = region.centroid
        ax.text(x, y, f'{region.label}', color='red', fontsize=12, ha='center')
    ax.axis('off')
    plt.close(fig_numbered)

    # Calculating properties of refined labeled regions
    properties = []
    for region in regionprops(final_labels):
        # Compute additional properties for each nucleus
        area = region.area
        perimeter = region.perimeter
        compactness = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0
        eccentricity = region.eccentricity
        solidity = region.solidity
        major_axis_length = region.major_axis_length
        minor_axis_length = region.minor_axis_length
        aspect_ratio = major_axis_length / minor_axis_length if minor_axis_length > 0 else 0
        orientation = region.orientation

        # Add computed properties to the properties list
        properties.append({
            "area": area,
            "perimeter": perimeter,
            "compactness": compactness,
            "eccentricity": eccentricity,
            "solidity": solidity,
            "major_axis_length": major_axis_length,
            "minor_axis_length": minor_axis_length,
            "aspect_ratio": aspect_ratio,
            "orientation": orientation,
        })

    # Generate individual masks for gallery
    individual_masks = [(final_labels == i).astype(np.uint8) * 255 for i in range(1, final_labels.max() + 1)]

    return fig_colored, fig_numbered, individual_masks, properties

# Function to fetch properties of a selected nucleus
def get_nucleus_properties(index, properties):
    if 0 <= index < len(properties):
        selected_props = properties[index]
        return (
            f"Area: {selected_props['area']} pixels\n"
            f"Perimeter: {selected_props['perimeter']:.2f} pixels\n"
            f"Compactness: {selected_props['compactness']:.2f}\n"
            f"Eccentricity: {selected_props['eccentricity']:.2f}\n"
            f"Solidity: {selected_props['solidity']:.2f}\n"
            f"Major Axis Length: {selected_props['major_axis_length']:.2f} pixels\n"
            f"Minor Axis Length: {selected_props['minor_axis_length']:.2f} pixels\n"
            f"Aspect Ratio: {selected_props['aspect_ratio']:.2f}\n"
            f"Orientation: {selected_props['orientation']:.2f} radians"
        )
    else:
        return "Select a nucleus to view its properties."

# New function for drawing on an image and calculating shape properties
def create_filled_binary_mask_and_calculate_properties(drawn_shape):
    debug_output = io.StringIO()
    try:
        if isinstance(drawn_shape, dict) and 'layers' in drawn_shape:
            drawn_shape = drawn_shape['layers']

        if drawn_shape is None:
            raise ValueError("No drawing layer found in the dictionary returned by Sketchpad")

        if not isinstance(drawn_shape, np.ndarray):
            drawn_shape = np.array(drawn_shape, dtype=np.uint8)

        if drawn_shape.ndim == 4 and drawn_shape.shape[0] == 1:
            drawn_shape = np.squeeze(drawn_shape, axis=0)

        if drawn_shape.ndim == 3 and drawn_shape.shape[2] == 4:
            alpha_channel = drawn_shape[:, :, 3]
            mask = alpha_channel > 0
            gray_image = np.where(mask, 255, 0).astype(np.uint8)
        elif drawn_shape.ndim == 3:
            gray_image = cv2.cvtColor(drawn_shape, cv2.COLOR_RGB2GRAY)
        else:
            gray_image = drawn_shape

        _, binary_mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filled_mask = np.zeros_like(binary_mask)
        cv2.drawContours(filled_mask, contours, -1, color=255, thickness=cv2.FILLED)

        if not contours:
            return filled_mask, "No contours found."

        contour = max(contours, key=cv2.contourArea)

        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        compactness = (perimeter**2) / (4 * np.pi * area) if area > 0 else 0

        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        solidity = area / hull_area if hull_area > 0 else 0

        if len(contour) >= 5:
            ellipse = cv2.fitEllipse(contour)
            (center, axes, angle) = ellipse
            major_axis_length = max(axes)
            minor_axis_length = min(axes)
            aspect_ratio = major_axis_length / minor_axis_length if minor_axis_length > 0 else 0
            orientation = np.deg2rad(angle)
            eccentricity = np.sqrt(1 - (minor_axis_length / major_axis_length)**2)
        else:
            major_axis_length = minor_axis_length = aspect_ratio = orientation = eccentricity = 0

        properties_text = (
            f"Area: {area} pixels\n"
            f"Perimeter: {perimeter:.2f} pixels\n"
            f"Compactness: {compactness:.2f}\n"
            f"Eccentricity: {eccentricity:.2f}\n"
            f"Solidity: {solidity:.2f}\n"
            f"Major Axis Length: {major_axis_length:.2f}\n"
            f"Minor Axis Length: {minor_axis_length:.2f}\n"
            f"Aspect Ratio: {aspect_ratio:.2f}\n"
            f"Orientation: {orientation:.2f} radians"
        )

        return filled_mask, properties_text
    except Exception as e:
        return np.zeros((512, 512), dtype=np.uint8), f"Error: {e}"

'''
# Gradio Interface for Segmenting Cells
application_interface = gr.Interface(
    fn=segment_cells,
    inputs=gr.Image(label="Upload an Image", type="numpy"),
    outputs=[
        gr.Plot(label="Refined Segmentation Mask"),
        gr.Plot(label="Numbered Nuclei Mask"),
        gr.Gallery(label="Individual Nuclei Masks", interactive=True),
        gr.Textbox(label="Nucleus Properties", placeholder="Select a nucleus to view its properties")
    ],
    title="Cell Segmentor Dashboard - Application",
    description="Upload an image of cells. The model will segment the nuclei with refined watershed-based separation for overlapping objects, and label each nucleus with a unique number.",
)
'''
with gr.Blocks() as application_interface:
    gr.Markdown("# Cell Segmentation with U-Net++ and Refined Post-Processing")

    with gr.Row():
        input_image = gr.Image(label="Upload an Image", type="numpy")

    with gr.Row():
        segmented_colored = gr.Plot(label="Refined Segmentation Mask")
        segmented_numbered = gr.Plot(label="Numbered Binary Mask")

    individual_nuclei = gr.Gallery(label="Individual Nuclei Masks", interactive=True)
    nucleus_properties = gr.Textbox(label="Nucleus Properties", value="Select a nucleus to view its properties.")

    # Store properties as a state variable
    properties_state = gr.State()

    # Load image and perform segmentation
    def on_segment(image):
        fig_colored, fig_numbered, individual_masks, properties = segment_cells(image)
        return fig_colored, fig_numbered, individual_masks, properties

    # Fetch properties when an image is selected
    def on_select_image(evt: gr.SelectData, properties):
        selected_index = evt.index
        return get_nucleus_properties(selected_index, properties)

    # Connect functions to components
    input_image.upload(on_segment, inputs=input_image, outputs=[segmented_colored, segmented_numbered, individual_nuclei, properties_state])
    individual_nuclei.select(on_select_image, inputs=properties_state, outputs=nucleus_properties)

# Gradio Interface for Drawing on Image and Calculating Shape Properties
with gr.Blocks() as manual_separation_interface:
    gr.Markdown("## Upload an Image as Background, Draw on Transparent Layer, and Calculate Shape Properties")

    background_image_input = gr.Image(type="pil", label="Upload Background Image")
    drawn_shape_input = gr.Sketchpad(label="Draw on Layer", type="pil")

    filled_shape_output = gr.Image(label="Filled Shape")
    debug_output = gr.Textbox(label="Shape Properties and Debugging Output")

    generate_button = gr.Button("Generate segmented nucleus and relevant dimensions")
    generate_button.click(fn=create_filled_binary_mask_and_calculate_properties, inputs=[drawn_shape_input], outputs=[filled_shape_output, debug_output])

    background_image_input.change(fn=lambda img: img.convert("RGBA"), inputs=[background_image_input], outputs=[drawn_shape_input])

# Combine both interfaces into a tabbed interface
app_interface = gr.TabbedInterface([application_interface, manual_separation_interface], ["Automatic Segmentation", "Manual Segmentation"])
app_interface.launch(share=True)


<ipython-input-83-a73c8a689ac4>:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e4a67503df61a473d2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--------------------------------------------------

---



---



About Page

In [78]:
import gradio as gr

def about_page():
    return gr.Markdown(
        """
        # About the Cell Segmentor Dashboard
        The **Cell Segmentor Dashboard** is a powerful tool developed for the segmentation of cell nuclei from microscopy images. It leverages a U-Net++ architecture for precise detection and separation of individual nuclei, making it especially useful for researchers in cellular biology.

        ### Model and Dataset
        - **Model Architecture**: The dashboard uses a U-Net++ model, a deep convolutional neural network designed for image segmentation tasks. U-Net++ improves on traditional U-Net by adding nested convolutions and skip connections, enabling better segmentation accuracy, particularly for complex and overlapping cell structures.
        - **Training Dataset**: The model was trained on the **Data Science Bowl 2018** dataset, which contains a diverse collection of cell images. The dataset was preprocessed to enhance contrast and resize images, making them suitable for the segmentation tasks in this application.

        ### Training and Performance
        - **Training Process**: The model was trained using a combination of Binary Cross-Entropy and Dice loss, optimized with Focal Loss to handle class imbalances. Training used a cyclic learning rate schedule to achieve high accuracy.
        - **Evaluation Metrics**: Model performance is measured using the Dice coefficient and Intersection over Union (IoU) metrics. After training, the model achieved:
          - **Mean Dice Score**: 0.8666
          - **Mean IoU Score**: 0.7727
        - **Early Stopping**: To prevent overfitting, early stopping was applied based on the Dice score.

        ### Key Functionalities
        - **Automated Segmentation**: Upload a microscopy image, and the model will automatically segment the cell nuclei. It employs watershed and distance transform techniques to refine segmentation and separate overlapping nuclei.
        - **Visualization**: View the segmentation mask with individual nuclei labeled, allowing for easy identification and selection.
        - **Property Analysis**: For each segmented nucleus, detailed properties are calculated, including area, perimeter, eccentricity, and solidity. This information helps in quantitative morphological analysis of the nuclei.
        - **Manual Drawing Tool**: Users can also draw custom shapes on an image for a more focused analysis of specific areas. Shape properties are calculated for each custom region.

        ### How to Use
        - **Upload an Image**: Navigate to the **Segment Cells** tab to upload a microscopy image. The tool will then process the image and display the refined segmentation results.
        - **Interactive Nucleus Selection**: Each nucleus is labeled, and users can select individual nuclei to view their properties.
        - **Manual Drawing**: Use the **Draw and Calculate Shape Properties** tab to upload an image and manually draw shapes for custom analysis.

        ### Evaluation Example
        Below is an example of the training and validation performance plot showing both the loss and Dice coefficient over epochs. This demonstrates how the model's accuracy improved over time.

        ![Training and Validation Loss and Dice Coefficient](https://raw.githubusercontent.com/desstaw/CellSeg/refs/heads/main/train_val.png)

        ### Acknowledgments
        This project was developed using open-source libraries like PyTorch, Torchvision, and Albumentations, alongside data provided by the Data Science Bowl. Special thanks to the open-source community and contributors to these libraries, making tools like this possible for advancing cell biology research.
        """
    )

# Create the about page layout
about_interface = gr.Interface(
    fn=about_page,
    inputs=[],
    outputs="markdown",
    title="Cell Segmentor Dashboard - About"
)

about_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e726a853f8abbb396.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Combine pages into a tabbed interface

In [81]:
# Define the combined Gradio tabbed interface
dashboard = gr.TabbedInterface(
    [home_interface, app_interface, about_interface],
    ["Home", "Application", "About"]
)

# Launch the complete dashboard
dashboard.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0e059be4ad88f16a96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
